https://www.youtube.com/watch?v=bD6V3rcr_54


In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

環境

In [ ]:
# Cutom Env
# inherit from openAI Env
class ShowEnv(Env):
  # initialize
  def __init__(self):
    # actionの取りうる値の空間
    # 0: down, 1: stay, 2: up
    self.action_space = Discrete(3)
    # 観測データの取りうる値の空間: 
    # continuous (multiple-value) value.  Used for images dataframe 
    self.observation_space =  Box(low=np.array([0]), high=np.array([100]))
    # state: temprature
    self.state = 38+ random.randint(-1,3)
    # iteration
    self.shower_length = 60

  def step(self, action):
    # Apply action
    # 0 -1 = -1 temperature
    # 1 -1 = 0 
    # 2 -1 = 1 temperature 
    self.state += action -1 
    # Reduce shower length by 1 second
    self.shower_length -= 1 
    
    # Calculate reward
    # very simple
    if self.state >=37 and self.state <=39: 
        reward =1 
    else: 
        reward = -1 
    
    # Check if shower is done ( check finish )
    if self.shower_length <= 0: 
        done = True
    else:
        done = False
    
    # Apply temperature noise
    # Havinng nonise is aid to simulate a real env
    # but it takes longer 
    self.state += random.randint(-1,1)
    # Set placeholder for info
    info = {}
    
    # Return step information
    return self.state, reward, done, info


  # visualize
  def render(self):
    # imfomative
    pass

  # reset env
  def reset(self):
      # Reset shower temperature
      self.state = 38 + random.randint(-3,3)
      # Reset shower time
      self.shower_length = 60 
      return self.state

In [ ]:
# Get first env
env = ShowEnv()

In [ ]:
# Get first env
env = ShowEnv()
# temperature example　
print(env.observation_space.sample())
# Action example
print(env.action_space.sample())

In [ ]:
# 試しに２５回してみる
episodes =  25
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

DQNで用いるDNNモデル

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# Create Simple DNN
def build_model(states, actions):
  model = Sequential()
  model.add(Dense(24,activation = 'relu', input_shape=states))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(actions, activation='linear')) #return action
  return model

In [ ]:
model = build_model(
    states = env.observation_space.shape,
    actions = env.action_space.n
)

In [ ]:
model.summary()

Agent

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
  policy = BoltzmannQPolicy()
  memory = SequentialMemory(limit=50000, window_length=1)
  # DQNのエージェント
  # model: 上のDNN
  # memory: Experience Replay用のメモリ。特定の順序で呼び出すSequentialMemory
  # policy: 行動を選択ルール(方策)。BoltzmannQPolicyは行動のQ値をソフトマックスにかけて選択する

  dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
  return dqn

In [ ]:
dqn = build_agent(
    model = model,
    actions = env.action_space.n
)
dqn.compile(Adam(lr=1e-3),  metrics=['mae'])#lossは設定不可
dqn.fit(env, nb_steps=500, visualize=False, verbose=1)

testする。


In [ ]:
import rl.callbacks
# call back も設定できる
class EpisodeLogger(rl.callbacks.Callback):
    def __init__(self):
        self.observations = {}
        self.rewards = {}
        self.actions = {}

    def on_episode_begin(self, episode, logs):
        self.observations[episode] = []
        self.rewards[episode] = []
        self.actions[episode] = []

    def on_step_end(self, step, logs):
        episode = logs['episode']
        self.observations[episode].append(logs['observation'])
        self.rewards[episode].append(logs['reward'])
        self.actions[episode].append(logs['action'])
cb_ep = EpisodeLogger()

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=False, callbacks=[cb_ep])
print(np.mean(scores.history['episode_reward']))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

for obs in cb_ep.observations.values():
  plt.plot(obs)
plt.xlabel("step")
plt.ylabel("pos")


In [ ]:
# https://qiita.com/goodclues/items/9b2b618ac5ba4c3be1c5#%E7%92%B0%E5%A2%83dnn%E3%83%A2%E3%83%87%E3%83%AB

In [ ]:
#renderを実装すれば行ける
# _ = dqn.test(env, nb_episodes=15, visualize=True)

In [ ]:
# save
dqn.save_weights('dqn_weights.h5f', overwrite=True)
# del model
# del dqn
# del env

In [ ]:
import gym
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)